# LSTM

In [1]:
import os
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

from utils.dataset import BabyMotionDataset
from models.lstm import LSTMClassifier
from utils.utils_func import collate_fn

In [ ]:
# params
num_epoch = 10000
lr = 1e-3

use_aug = True
aug_method = 'GaussianNoise'
val_method = 'lstm'

In [ ]:
origin_dir = "./data_origin/sequence"
aug_dirs = os.path.join("./data_aug", aug_method) if use_aug else None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

label2idx = {
    'crawl': 0, 'walk': 1,
    'sit-floor': 2, 'sit-high-chair': 3, 'sit-low-chair': 4, 'stand': 5, 
    'hold-horizontal': 6, 'hold-vertical': 7, 'piggyback': 8, 
    'baby-food': 9, 'bottle': 10, 'breast': 11, 
    'face-down': 12, 'face-side': 13, 'face-up':14, 'roll-over': 15
}

# logger
now = datetime.now().strftime("%Y%m%d_%H%M%S")
log_root = f"./logs/{val_method}/{aug_method if use_aug else "origin"}/{now}"
checkpoints_root = os.path.join(log_root, "checkpoints")
os.makedirs(log_root, exist_ok=True)
os.makedirs(checkpoints_root, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(log_root, "train.log"),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [ ]:
train_dataset = BabyMotionDataset(
    origin_dir=origin_dir,
    aug_dirs=aug_dirs,
    max_len=100,
    min_len=10,
    is_train=True
)
val_dataset = BabyMotionDataset(
    origin_dir=origin_dir,
    aug_dirs=aug_dirs,
    max_len=100,
    min_len=10,
    is_train=False
)

# DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=lambda b: collate_fn(b, label2idx)
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=lambda b: collate_fn(b, label2idx)
)

In [5]:
# 验证函数
def evaluate(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, lengths, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x, lengths)
            preds = out.argmax(dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.cpu().tolist())
    acc = accuracy_score(all_labels, all_preds)
    return acc

In [6]:
model = LSTMClassifier(input_dim=3, hidden_dim=64, num_classes=len(label2idx)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

best_val_acc = 0.0
for epoch in range(num_epoch):
    model.train()
    all_preds, all_labels = [], []

    for batch in train_loader:
        x, lengths, y = batch
        x, y = x.to(device), y.to(device)

        output = model(x, lengths)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = output.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y.cpu().numpy())

    train_acc = accuracy_score(all_labels, all_preds)
    val_acc = evaluate(model, val_loader)

    log_msg = f"[Epoch {epoch}] Loss: {loss.item():.9f} | Train Acc: {train_acc:.9f} | Val Acc: {val_acc:.9f}"
    print(log_msg)
    logging.info(log_msg)

    # save model at best val acc.
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(checkpoints_root, f"best_model_epoch{epoch}.pt"))
        logging.info(f"[Epoch {epoch}] Saved new best model with Val Acc: {val_acc:.9f}")

[Epoch 0] Loss: 2.598090410 | Train Acc: 0.124324324 | Val Acc: 0.200000000
[Epoch 1] Loss: 2.104791641 | Train Acc: 0.206756757 | Val Acc: 0.381818182
[Epoch 2] Loss: 1.563715458 | Train Acc: 0.336486486 | Val Acc: 0.272727273
[Epoch 3] Loss: 1.875556827 | Train Acc: 0.333783784 | Val Acc: 0.300000000
[Epoch 4] Loss: 1.940047264 | Train Acc: 0.378378378 | Val Acc: 0.363636364
[Epoch 5] Loss: 1.791242599 | Train Acc: 0.429729730 | Val Acc: 0.400000000
[Epoch 6] Loss: 2.231987000 | Train Acc: 0.383783784 | Val Acc: 0.290909091
[Epoch 7] Loss: 1.620212317 | Train Acc: 0.420270270 | Val Acc: 0.390909091
[Epoch 8] Loss: 1.496049762 | Train Acc: 0.497297297 | Val Acc: 0.418181818
[Epoch 9] Loss: 1.676199555 | Train Acc: 0.472972973 | Val Acc: 0.472727273
[Epoch 10] Loss: 0.958085120 | Train Acc: 0.505405405 | Val Acc: 0.527272727
[Epoch 11] Loss: 1.291100264 | Train Acc: 0.510810811 | Val Acc: 0.518181818
[Epoch 12] Loss: 1.133111238 | Train Acc: 0.518918919 | Val Acc: 0.481818182
[Epoch 13